# Search Engine with Tools and Agents

In [1]:
## Arxiv -- Research

## Tools creation

from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun

from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper


In [2]:
## Used the inbuilt tool for wikipedia

api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)

wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)

wiki.name

'wikipedia'

In [3]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper = api_wrapper_arxiv)

print(arxiv.name)

arxiv


In [4]:
tools = [wiki,arxiv]

In [5]:
## Custom tools [RAG Tool]

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
#from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\xpert computers\AppData\Local\Temp\ipykernel_5164\3468779532.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\xpert computers\Desktop\class 31\Class31\my_new_env10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")

docs= loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

vectordb = FAISS.from_documents(documents,embeddings)

retriever = vectordb.as_retriever()

retriever


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020644E3B640>, search_kwargs={})

In [7]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever = retriever,
    name= "langsmith_search",
    description = "Search any information about Langchain")

retriever_tool.name

'langsmith_search'

In [8]:
tools = [wiki,arxiv,retriever_tool]

In [9]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\xpert computers\\Desktop\\class 31\\Class31\\my_new_env10\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith_search', description='Search any information about Langchain', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002062752D3F0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.F

### Conculsion to this :

- Tool Creation = How to build or retrieve tools from wikipedia or arxiv by installing specific libraries and wrapping them in Langchain's convenience classes.

- Custom Tools = How to load your own data ( like a webpage) and turn it into a retriever, then wrap that retriever as a tool.

- Preparing for Agents = We're setting the stage so that in the next part we'll integrate these tools with an LLM or Agen. The agent can automatiocally pick which tool to use based on the question, giving a more powerful "search engine" experience


In [13]:
## Run all this tools with Agents and LLM Models

## Tools, LLM -- > AgentExecutor


from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
import os

groq_api_key = os.getenv("GROQ_API_KEY")

huggingface_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")


In [14]:
from langchain import hub
prompt= hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Users\xpert computers\Desktop\class 31\Class31\my_new_env10\lib\site-packages\langsmith\client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [16]:
from langchain.agents import create_tools_agent

agent = create_tools_agent(llm=llm, tools=tools, prompt=prompt)

ImportError: cannot import name 'create_tools_agent' from 'langchain.agents' (c:\Users\xpert computers\Desktop\class 31\Class31\my_new_env10\lib\site-packages\langchain\agents\__init__.py)

In [ ]:
## Agent Executor

